In [1]:
import pandas as pd
pd.set_option('max_columns', 180)
pd.set_option('max_rows', 200000)
pd.set_option('max_colwidth', 5000)

game_log = pd.read_csv("game_log.csv")
park_codes = pd.read_csv("park_codes.csv")
person_codes = pd.read_csv("person_codes.csv")
team_codes = pd.read_csv("team_codes.csv")

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (12,13,14,15,19,20,81,82,83,84,85,86,87,88,93,94,95,96,97,98,99,100,105,106,108,109,111,112,114,115,117,118,120,121,123,124,126,127,129,130,132,133,135,136,138,139,141,142,144,145,147,148,150,151,153,154,156,157,160) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
import sqlite3
def run_query(q):
    with sqlite3.connect('mlb.db') as conn:
        return pd.read_sql(q, conn)

def run_command(q):
    with sqlite3.connect('mlb.db') as conn:
        conn.execute('PRAGMA foreign_keys = ON;')
        conn.isolation_level = None
        return conn.execute(q)

In [3]:
conn = sqlite3.connect('mlb.db')
game_log.to_sql('game_log', conn, index=False)
park_codes.to_sql('park_codes', conn, index=False)
person_codes.to_sql('person_codes', conn, index=False)
team_codes.to_sql('team_codes', conn, index=False)
conn.close()

In [4]:
# Reference - date, h_name, and number_of_game
q_add_gameid = "ALTER TABLE game_log ADD game_id TEXT"
run_command(q_add_gameid)

In [5]:
q_update_gameid = """
    UPDATE game_log
    SET game_id = h_name || date || number_of_game
"""
run_command(q_update_gameid)

In [6]:
q = """
    CREATE TABLE IF NOT EXISTS person (
        person_id text PRIMARY KEY,
        first_name text,
        last_name text
    );
"""
run_command(q)

In [7]:
q = """
    INSERT INTO person
    SELECT id, first, last FROM person_codes;
"""
q2 = "SELECT * FROM person LIMIT 5"
run_command(q)
run_query(q2)

,person_id,first_name,last_name
0,aardd001,David,Aardsma
1,aaroh101,Hank,Aaron
2,aarot101,Tommie,Aaron
3,aased001,Don,Aase
4,abada001,Andy,Abad


In [8]:
q = """
    CREATE TABLE IF NOT EXISTS park (
       park_id text PRIMARY KEY,
       name text,
       nickname text,
       city text,
       state text,
       notes textt
    );
"""
run_command(q)

In [9]:
q = """  
    INSERT INTO park
    SELECT
        park_id,
        name,
        aka,
        city,
        state,
        notes
    FROM park_codes
"""
q2 = "SELECT * FROM park LIMIT 5"
run_command(q)
run_query(q2)

,park_id,name,nickname,city,state,notes
0,ALB01,Riverside Park,None,Albany,NY,TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882
1,ALT01,Columbia Park,None,Altoona,PA,None
2,ANA01,Angel Stadium of Anaheim,Edison Field; Anaheim Stadium,Anaheim,CA,None
3,ARL01,Arlington Stadium,None,Arlington,TX,None
4,ARL02,Rangers Ballpark in Arlington,The Ballpark in Arlington; Ameriquest Fl,Arlington,TX,None


In [10]:
q = """
    CREATE TABLE IF NOT EXISTS league (
        league_id TEXT PRIMARY KEY,
        name TEXT
    )
"""
run_command(q)

In [11]:
q = """
    INSERT INTO league (league_id)
    VALUES ("None"), ("NL"), ("AA"), ("UA"), ("PL"), ("AL"), ("FL")
"""
q2 = "SELECT * FROM league LIMIT 10"
run_command(q)
run_query(q2)

,league_id,name
0,None,None
1,NL,None
2,AA,None
3,UA,None
4,PL,None
5,AL,None
6,FL,None


In [12]:
q = """
    CREATE TABLE IF NOT EXISTS appearance_type (
        appearance_type_id text PRIMARY KEY,
        name text,
        category text
    );
"""
run_command(q)

In [17]:
conn = sqlite3.connect('mlb.db')
appearance_data = pd.read_csv("appearance_type.csv")
appearance_data.to_sql('appearance_type', conn, index=False, if_exists='append')
q = "SELECT * FROM appearance_type LIMIT 10"
run_query(q)

,appearance_type_id,name,category
0,O1,Batter 1,offense
1,O2,Batter 2,offense
2,O3,Batter 3,offense
3,O4,Batter 4,offense
4,O5,Batter 5,offense
5,O6,Batter 6,offense
6,O7,Batter 7,offense
7,O8,Batter 8,offense
8,O9,Batter 9,offense
9,D1,Pitcher,defense


In [18]:
q = """
    CREATE TABLE IF NOT EXISTS team (
        team_id text PRIMARY KEY,
        league_id text,
        city text,
        nickname text,
        franch_id text,
        FOREIGN KEY(league_id) REFERENCES league(league_id)
    );
"""
run_command(q)

In [21]:
q = """
    INSERT INTO team
    SELECT 
        team_id, 
        league, 
        city, 
        nickname, 
        franch_id
    FROM team_codes
    GROUP BY team_id
"""
run_command(q)

In [26]:
q2 = "SELECT * FROM team LIMIT 10"
run_query(q2)

,team_id,league_id,city,nickname,franch_id
0,ALT,UA,Altoona,Mountain Cities,ALT
1,ANA,AL,Anaheim,Angels,LAA
2,ARI,NL,Arizona,Diamondbacks,ARI
3,ATL,NL,Atlanta,Braves,BS1
4,BAL,AL,Baltimore,Orioles,MLA
5,BFN,NL,Buffalo,Bisons,BFN
6,BFP,PL,Buffalo,Bisons,BFP
7,BL1,None,Baltimore,Canaries,BL1
8,BL2,AA,Baltimore,Orioles,BL2
9,BL4,None,Baltimore,Marylands,BL4


In [30]:
q = """
    CREATE TABLE IF NOT EXISTS game (
        game_id text PRIMARY KEY,
        date integer,
        number_of_game integer,
        park_id text,
        length_outs real,
        day integer,
        completion text,
        forefeit text, 
        protest text,
        attendance real,
        length_minutes real,
        additional_info text,
        acquisition_info text,
        FOREIGN KEY(park_id) REFERENCES park(park_id)
    )
""" 
run_command(q)

In [31]:
q = """
    INSERT INTO game
    SELECT
        game_id,
        date,
        number_of_game,
        park_id,
        length_outs,
        day_night,
        completion,
        forefeit,
        protest,
        attendance,
        length_minutes,
        additional_info,
        acquisition_info
    FROM game_log
"""
q2 = "SELECT * FROM game LIMIT 10"
run_command(q)
run_query(q2)

,game_id,date,number_of_game,park_id,length_outs,day,completion,forefeit,protest,attendance,length_minutes,additional_info,acquisition_info
0,FW1187105040,18710504,0,FOR01,54.0,D,None,None,None,200.0,120.0,None,Y
1,WS3187105050,18710505,0,WAS01,54.0,D,None,None,None,5000.0,145.0,HTBF,Y
2,RC1187105060,18710506,0,RCK01,54.0,D,None,None,None,1000.0,140.0,None,Y
3,CH1187105080,18710508,0,CHI01,54.0,D,None,None,None,5000.0,150.0,None,Y
4,TRO187105090,18710509,0,TRO01,54.0,D,None,None,None,3250.0,145.0,HTBF,Y
5,CL1187105110,18710511,0,CLE01,48.0,D,None,V,None,2500.0,120.0,None,Y
6,CL1187105130,18710513,0,CIN01,54.0,D,None,None,None,1200.0,150.0,None,Y
7,FW1187105130,18710513,0,FOR01,54.0,D,None,None,None,1500.0,105.0,None,Y
8,FW1187105150,18710515,0,FOR01,54.0,D,None,None,None,NaN,140.0,None,Y
9,BS1187105160,18710516,0,BOS01,54.0,D,None,None,None,2500.0,NaN,HTBF,Y


In [41]:
q = """
    CREATE TABLE IF NOT EXISTS team_appearance (
        team_id text,
        game_id text,
        home integer,
        league_id text,
        score integer,
        line_score text,
        at_bats real,
        hits real,
        doubles real,
        triples real,
        homeruns real,
        rbi real,
        sacrifice_hits real,
        sacrifice_flies real,
        hit_by_pitch real,
        walks real,
        intentional_walks real,
        strikeouts real,
        stolen_bases real,
        caught_stealing real,
        grouded_into_double real,
        first_catcher_interference real,
        left_on_base real,
        pitchers_used real,
        individual_earned_runs real,
        team_earned_runs real,
        wild_pitches real,
        balks real,
        putouts real,
        assists real,
        errors real,
        passed_balls real,
        double_plays real,
        triple_plays real,
        FOREIGN KEY(team_id) REFERENCES team(team_id),
        FOREIGN KEY(game_id) REFERENCES game(game_id),
        FOREIGN KEY(league_id) REFERENCES league(league_id),
        PRIMARY KEY (team_id, game_id)
    )
"""
run_command(q)

In [42]:
q = """
    INSERT INTO team_appearance
        SELECT
            h_name,
            game_id,
            1 AS home,
            h_league,
            h_score,
            h_line_score,
            h_at_bats,
            h_hits,
            h_doubles,
            h_triples,
            h_homeruns,
            h_rbi,
            h_sacrifice_hits,
            h_sacrifice_flies,
            h_hit_by_pitch,
            h_walks,
            h_intentional_walks,
            h_strikeouts,
            h_stolen_bases,
            h_caught_stealing,
            h_grounded_into_double,
            h_first_catcher_interference,
            h_left_on_base,
            h_pitchers_used,
            h_individual_earned_runs,
            h_team_earned_runs,
            h_wild_pitches,
            h_balks,
            h_putouts,
            h_assists,
            h_errors,
            h_passed_balls,
            h_double_plays,
            h_triple_plays
        FROM game_log
    UNION
        SELECT
            v_name,
            game_id,
            0 AS home,
            v_league,
            v_score,
            v_line_score,
            v_at_bats,
            v_hits,
            v_doubles,
            v_triples,
            v_homeruns,
            v_rbi,
            v_sacrifice_hits,
            v_sacrifice_flies,
            v_hit_by_pitch,
            v_walks,
            v_intentional_walks,
            v_strikeouts,
            v_stolen_bases,
            v_caught_stealing,
            v_grounded_into_double,
            v_first_catcher_interference,
            v_left_on_base,
            v_pitchers_used,
            v_individual_earned_runs,
            v_team_earned_runs,
            v_wild_pitches,
            v_balks,
            v_putouts,
            v_assists,
            v_errors,
            v_passed_balls,
            v_double_plays,
            v_triple_plays
        FROM game_log;
"""
q2 = "SELECT * FROM team_appearance LIMIT 40"
run_command(q)
run_query(q2)

,team_id,game_id,home,league_id,score,line_score,at_bats,hits,doubles,triples,homeruns,rbi,sacrifice_hits,sacrifice_flies,hit_by_pitch,walks,intentional_walks,strikeouts,stolen_bases,caught_stealing,grouded_into_double,first_catcher_interference,left_on_base,pitchers_used,individual_earned_runs,team_earned_runs,wild_pitches,balks,putouts,assists,errors,passed_balls,double_plays,triple_plays
0,ALT,ALT188404300,1,UA,2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,ALT,ALT188405020,1,UA,3,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,ALT,ALT188405030,1,UA,5,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,ALT,ALT188405050,1,UA,2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,ALT,ALT188405100,1,UA,9,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
5,ALT,ALT188405120,1,UA,3,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
6,ALT,ALT188405140,1,UA,2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
7,ALT,ALT188405150,1,UA,7,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
8,ALT,ALT188405160,1,UA,6,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
9,ALT,ALT188405170,1,UA,8,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [47]:
q = "SELECT * FROM team_appearance LIMIT 4"
run_query(q)

,team_id,game_id,home,league_id,score,line_score,at_bats,hits,doubles,triples,homeruns,rbi,sacrifice_hits,sacrifice_flies,hit_by_pitch,walks,intentional_walks,strikeouts,stolen_bases,caught_stealing,grouded_into_double,first_catcher_interference,left_on_base,pitchers_used,individual_earned_runs,team_earned_runs,wild_pitches,balks,putouts,assists,errors,passed_balls,double_plays,triple_plays
0,ALT,ALT188404300,1,UA,2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,ALT,ALT188405020,1,UA,3,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,ALT,ALT188405030,1,UA,5,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,ALT,ALT188405050,1,UA,2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
